In [ ]:
# GPU 조절
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
# os.environ["GRADIO_SHARE"]="3"
# os.environ["WORLD_SIZE"] = "3"
# os.getcwd()
import torch
torch.cuda.device_count() #must be 1

In [2]:
BASE_MODEL_NAME = "meetkai/functionary-small-v3.2"
MAX_SEQ_LENGTH = 2048 # Choose any! We auto support RoPE Scaling internally!
DTYPE = None #Auto
LOAD_IN_4BIT = False
LORA_R = 64 # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
LORA_ALPHA = 64
LORA_RANDOM_STATE = 148177255
EXCEL_PATH = '~/S11P21B109/backend/fastapi_gpu/finetuning/law_data.xlsx'
TRAINER_BATCH=16
TRAINER_ACCUMULATION=4
TRAINER_EPOCH=2
TRAINER_LR = 1e-4
SAVE_SETTING = "merged_16bit"

In [ ]:
from unsloth import FastLanguageModel
import torch
# padding 없는 건 정상
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL_NAME,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = DTYPE,
    load_in_4bit = LOAD_IN_4BIT,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = LORA_R, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = LORA_ALPHA,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = LORA_RANDOM_STATE,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [5]:
import pandas as pd
from datasets import Dataset

# Step 1: Load the XLSX file into a pandas DataFrame
df = pd.read_excel(EXCEL_PATH)  

df = df.rename(columns={
    'Prompt': 'instruction',
    'User': 'input',
    'Assistant': 'output'
})
df = df[['instruction', 'input', 'output']]

# Convert columns to 'string' dtype
df['instruction'] = df['instruction'].astype('string')
df['input'] = df['input'].astype('string')
df['output'] = df['output'].astype('string')

df.reset_index(drop=True, inplace=True)
data_dict = df.to_dict('records')

# Step 3: Convert the DataFrame into a Hugging Face Dataset
dataset = Dataset.from_list(data_dict)

In [6]:
#SPECIAL TOKENS FOR LLAMA 3.1
ROLE_SYSTEM_TOKEN = "system"
ROLE_USER_TOKEN= "user"
ROLE_ASSISTANT_TOKEN = "assistant"
SH_TOKEN = "<|start_header_id|>"
EH_TOKEN = "<|end_header_id|>"
EOS_TOKEN = tokenizer.eos_token

#for prompts
DOUBLE_NEWLINE = '\n\n'

#Meekai/functionary token

FUNCTIONARY_FUNCTION_SEPARATOR = ">>>"
FUNCTIONARY_PLAIN_TOKEN = 'all\n'

In [ ]:
from trl import DataCollatorForCompletionOnlyLM #for completion

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs): #double newline 이미 있음
        text = SH_TOKEN + ROLE_SYSTEM_TOKEN + EH_TOKEN + instruction + EOS_TOKEN\
        + SH_TOKEN + ROLE_USER_TOKEN + EH_TOKEN + DOUBLE_NEWLINE + input + EOS_TOKEN\
        + SH_TOKEN + ROLE_ASSISTANT_TOKEN + EH_TOKEN + DOUBLE_NEWLINE + FUNCTIONARY_FUNCTION_SEPARATOR + FUNCTIONARY_PLAIN_TOKEN +output + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

from datasets import load_dataset
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset_text = dataset.remove_columns([col for col in dataset.column_names if col != "text"]) 

In [8]:
from datasets import load_dataset, concatenate_datasets, Dataset

ds = load_dataset("openai/MMMLU", "by_language")

def formatting_prompts_func_qa(examples):
    AT = {
        "A" : "1",
        "B" : "2",    
        "C" : "3",
        "D" : "4",
    }
    qa_instruction = "You are a student AI studying to become a helpful assistant. you will be given a korean question. solve it. answer choice number only."    
    questions = examples["Question"]
    aas = examples["A"]
    abs = examples["B"]
    acs = examples["C"]
    ads = examples["D"]
    answers = examples["Answer"]
    texts = []
    for question, aa, ab, ac, ad, answer in zip(questions, aas, abs, acs, ads, answers):
        text = SH_TOKEN + ROLE_SYSTEM_TOKEN + EH_TOKEN + DOUBLE_NEWLINE +qa_instruction + EOS_TOKEN\
        + SH_TOKEN + ROLE_USER_TOKEN + EH_TOKEN + DOUBLE_NEWLINE + question + "\n" + f"1:{aa}\n2:{ab}\n3:{ac}\n4:{ad}" + EOS_TOKEN\
        + SH_TOKEN + ROLE_ASSISTANT_TOKEN + EH_TOKEN + DOUBLE_NEWLINE + FUNCTIONARY_FUNCTION_SEPARATOR + FUNCTIONARY_PLAIN_TOKEN + AT[answer] + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

dataset_qa = ds["KO_KR"].map(formatting_prompts_func_qa, batched = True,)
dataset_qa_text = dataset_qa.remove_columns([col for col in dataset_qa.column_names if col != "text"]) 

In [10]:
merged_dataset = concatenate_datasets([dataset_qa_text, dataset_text])
merged_dataset = merged_dataset.shuffle(seed=LORA_RANDOM_STATE)

In [11]:
# dataset collator
collator = DataCollatorForCompletionOnlyLM(SH_TOKEN + ROLE_ASSISTANT_TOKEN + EH_TOKEN, tokenizer=tokenizer)

In [ ]:
merged_dataset[7000]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = merged_dataset,
    dataset_text_field = "text",
    max_seq_length = MAX_SEQ_LENGTH,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = TRAINER_BATCH,
        gradient_accumulation_steps = TRAINER_ACCUMULATION,
        warmup_steps = 5,
        num_train_epochs = TRAINER_EPOCH, # Set this for 1 full training run.
        learning_rate = TRAINER_LR,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = LORA_RANDOM_STATE,
        output_dir = "outputs",
    ),
    data_collator=collator,
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# 모델 저장. https://github.com/unslothai/unsloth/wiki 의 Manually saving to GGUF 참조
model.save_pretrained_merged("merged_model", tokenizer, save_method = SAVE_SETTING)

In [ ]:
# llama cpp 를 실행하는 conda enviroment로 실행
# git clone --recursive https://github.com/ggerganov/llama.cpp
# make clean -C llama.cpp
# make all -j -C llama.cpp
# pip install gguf protobuf

# python llama.cpp/convert_hf_to_gguf.py merged_model --outfile converted.gguf --outtype f16

In [ ]:
raise Exception("Don't go more")

In [ ]:
# Model output check
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
from transformers import TextStreamer

for datadict in dataset:
    inputs = tokenizer(
        ["<|start_header_id|>system<|end_header_id|>" + datadict["instruction"] + EOS_TOKEN\
        + "<|start_header_id|>user<|end_header_id|>" + datadict["input"] + EOS_TOKEN\
        + "<|start_header_id|>assistant<|end_header_id|>"
        ], return_tensors = "pt").to("cuda")
    text_streamer = TextStreamer(tokenizer)
    _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
